# Coordinated Multipoint Using a Quantum Computer
This notebook demonstrates

# What is Coordinated Multipoint?

TBD: add description, papers, etc

# Modeling a Network

TBD: explain the King's graph-based network model

In [ ]:
# This code imports from "helpers", a folder colocated with this Jupyter Notebook.
from helpers.network import configure_network

network, tx_over_rx, _, props = configure_network(lattice_size=3, ratio=0.75)
print(f"Graph stats: {props} \n Ratio: {tx_over_rx}")

In [ ]:
import matplotlib.pyplot as plt
from helpers.draw import draw_network
%matplotlib inline     


draw_network(network)

# Decoding Signals

TBD: explain linear filters